# ANALYSIS OF CHICAGO PUBLIC, CRIME, AND CENSUS DATA 

In [1]:
%load_ext sql

In [2]:
!pip install -q pandas==1.1.5

In [3]:
import csv, sqlite3, pandas

con = sqlite3.connect("FinalDB.db")
cur = con.cursor()

In [4]:
import pandas as pd
df1 = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv')
df2 = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv')
df3 = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv')
df1.to_sql('CENSUS_DATA', con, if_exists='replace', index=False, method="multi")
df2.to_sql('CHICAGO_PUBLIC_SCHOOLS', con, if_exists='replace', index=False, method="multi")
df3.to_sql('CHICAGO_CRIME_DATA', con, if_exists='replace', index=False, method="multi")

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  both result in 0.1234 being formatted as 0.12.


In [5]:
%sql sqlite:///FinalDB.db
%sql select * from CENSUS_DATA LIMIT 5;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


total number of crimes recorded in the CRIME table.


In [6]:
%sql select count(*) as total_crimes from CHICAGO_CRIME_DATA;

 * sqlite:///FinalDB.db
Done.


total_crimes
533


community areas with per capita income less than 11000.


In [7]:
%sql select community_area_name from CENSUS_DATA where per_capita_income <11000;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
West Garfield Park
South Lawndale
Fuller Park
Riverdale


all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)


In [8]:
%sql select case_number, description from CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%minor%';

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,DESCRIPTION
HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,ILLEGAL CONSUMPTION BY MINOR


all kidnapping crimes involving a child


In [9]:
%sql select * from CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%child%' and PRIMARY_TYPE = 'KIDNAPPING';

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


kinds of crimes that were recorded at schools


In [10]:
%sql select DISTINCT(PRIMARY_TYPE) from CHICAGO_CRIME_DATA WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%';

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


the average safety score for each type of school.


In [11]:
%sql select "Elementary, Middle, or High School", SUM(SAFETY_SCORE)/ COUNT(*) \
    as AVG_SAFETY_SCORE from CHICAGO_PUBLIC_SCHOOLS group by "Elementary, Middle, or High School";

 * sqlite:///FinalDB.db
Done.


"Elementary, Middle, or High School",AVG_SAFETY_SCORE
ES,44.696969696969695
HS,45.354838709677416
MS,48.0


5 community areas with highest % of households below poverty line


In [12]:
%sql select COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY  from CENSUS_DATA \
    ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


community area is most crime prone


In [13]:
%sql select community_area_number from (select community_area_number, max(total_crimes)\
    from (select community_area_number, count(*) as total_crimes \
    from chicago_crime_data where COMMUNITY_AREA_NUMBER != 'None' group by community_area_number));

 * sqlite:///FinalDB.db
Done.


community_area_number
25.0


Double-click **here** for a hint

<!--
Query for the 'community area number' that is most crime prone.
-->


the name of the community area with highest hardship index


In [14]:
%sql select community_area_name from (select community_area_name, max(hardship_index) from census_data);

 * sqlite:///FinalDB.db
Done.


community_area_name
Riverdale


the Community Area Name with most number of crimes?


In [15]:
%sql select community_area_name from census_data \
    WHERE community_area_number = (select community_area_number \
    from (select community_area_number, max(total_crimes) \
    from (select community_area_number, count(*) as total_crimes \
    from chicago_crime_data where COMMUNITY_AREA_NUMBER != 'None' group by community_area_number)));

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin
